# GP-MH implementations

In [2]:
import numpy as np
from scipy.stats import norm as norm


In [ ]:
# set seed

Below we have a generic MCMC for reference. Later we sort of implemented the MCMC.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Target distribution: standard normal distribution
def target_distribution(x):
    return np.exp(-0.5 * x**2) / np.sqrt(2 * np.pi)

# Metropolis-Hastings algorithm
def metropolis_hastings(target_dist, proposal_dist, proposal_sampler, initial_value, n_samples):
    samples = [initial_value]
    current_value = initial_value
    
    for _ in range(n_samples):
        proposed_value = proposal_sampler(current_value)
        acceptance_ratio = target_dist(proposed_value) / target_dist(current_value)
        
        if np.random.rand() < acceptance_ratio:
            current_value = proposed_value
        
        samples.append(current_value)
    
    return np.array(samples)

# Proposal distribution: symmetric normal distribution centered at the current state
def proposal_distribution(x, proposal_width=1.0):
    return np.random.normal(x, proposal_width)



    


In [ ]:
# Parameters
initial_value = 0
n_samples = 10000

# Run the Metropolis-Hastings algorithm
samples = metropolis_hastings(
    target_dist=target_distribution,
    proposal_dist=proposal_distribution,
    proposal_sampler=lambda x: proposal_distribution(x, proposal_width=1.0),
    initial_value=initial_value,
    n_samples=n_samples
)

# Plot the results
x = np.linspace(-5, 5, 1000)
plt.figure(figsize=(10, 5))
plt.hist(samples, bins=50, density=True, alpha=0.6, label='Sampled distribution')
plt.plot(x, target_distribution(x), label='Target distribution')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.title('Metropolis-Hastings Sampling')
plt.show()

In [4]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR


In [ ]:
# function to conduct initial log likelihood evaluations
def initial_data(t_init, traget_distn, prior_lower,prior_upper):
    logliks = np.zeros(t_init-1)
    thetas = np.zeros(t_init-1)
    for i in range(t_init):
        thetas[i] = np.random.uniform(lower=prior_lower,upper= prior_upper, size=1)
        logliks[i] = target_distn(thetas[i])
    return thetas, logliks

def epsilon_gamma(m,v,u):
    return norm.cdf(-np.abs(m-np.log(u))/np.sqrt(v))

def post_preds(theta_prop, theta_prev, gpr,prior):
    m,v=gpr.predict([theta_prev, theta_prop])
    return m[1]-m[0] + np.log(prior(theta_prop)/prior(theta_prev)),v[0,0] + v[1,1] - 2*v[0,1]



# we are assuming a prior which is uniform therefore we have upper and lower prior
def GP_MH(N,t_init, target_distn, prior_lower,prior_upper,proposal,epsilon):
    mh_samples=np.zeros(N)
    thetas,logliks_init=initial_data(t_init, target_distn, prior_lower,prior_upper)
    t = t_init-1
    gpr = GPR()
    gpr.fit(thetas,logliks_init)
    for i in range(1,N+1):
        theta_prop = proposal(mh_samples[i-1])
        u = np.random.uniform(0,1)
        # calculate post mean and post var
        m,v=post_preds(theta_prop,mh_samples[i-1], gpr,prior)

        while epsilon_gamma(m,v,u) > epsilon:
            ''''
                TODO: FIGURE OUT how to perform this optimisation
                theta_star = np.argmax()
                we are effectively using posterior predicitve and minimisng the variance of it 
            
            '''
            theta_star = 0
            loglik_star=target_distn(theta_star)
            # add to data
            # refit GP
            gpr=GPR()
            gpr.fit(thetas,logliks_init)
            # recalculate gamma_hat by predicting on thtea_prop and that prev sample
            m,v=post_preds(theta_prop,mh_samples[i-1], gpr,prior)

        # accept reject
        if m>=np.log(u):
            mh_samples[i]=theta_prop
        else:
            mh_samples[i]=mh_samples[i-1]
        


